# # 모듈
---

In [19]:
import open3d as o3d
import numpy as np
import torch
import pandas as pd
import re
import pickle as pkl
import json
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R

# # kitti datasets calib 확인
---

In [248]:
with open('/data/hwang/datasets/kitti/training/calib/000000.txt', 'r') as f:
    txt = f.readlines()

p0 = txt[0].replace('\n', '').split(' ')[1:] # 0번 카메라
p1 = txt[1].replace('\n', '').split(' ')[1:] # 1번 카메라
p2 = txt[2].replace('\n', '').split(' ')[1:] # 2번 카메라 (모델에 이 정보를 사용), (i, 4)위치는 기본 카메라와의 위상 차이 (각 x, y, z 축)
p3 = txt[3].replace('\n', '').split(' ')[1:] # 3번 카메라
R_rect = txt[4].replace('\n', '').split(' ')[1:] # 스테레오 카메라로 촬영 시 rotation matrix 보정 수치 (모노 카메라면 단위행렬 사용)
Tr_velo = txt[5].replace('\n', '').split(' ')[1:] # extrinsic
Tr_imu = txt[6].replace('\n', '').split(' ')[1:]

print(np.around(np.asarray(p0, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p1, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p2, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p3, dtype=float).reshape(-1, 4), 1), '\n')
print(np.asarray(R_rect, dtype=float).reshape(-1, 3), '\n')
print(np.asarray(Tr_velo, dtype=float).reshape(-1, 4), '\n')
print(np.asarray(Tr_imu, dtype=float).reshape(-1, 4), '\n')

[[707.   0. 604.   0.]
 [  0. 707. 181.   0.]
 [  0.   0.   1.   0.]] 

[[ 707.    0.  604. -380.]
 [   0.  707.  181.    0.]
 [   0.    0.    1.    0.]] 

[[707.   0. 604.  46.]
 [  0. 707. 181.  -0.]
 [  0.   0.   1.   0.]] 

[[ 707.    0.  604. -334.]
 [   0.  707.  181.    2.]
 [   0.    0.    1.    0.]] 

[[ 0.9999128   0.01009263 -0.00851193]
 [-0.01012729  0.9999406  -0.00403767]
 [ 0.00847068  0.00412352  0.9999556 ]] 

[[ 0.00692796 -0.9999722  -0.00275783 -0.02457729]
 [-0.00116298  0.00274984 -0.9999955  -0.06127237]
 [ 0.9999753   0.00693114 -0.0011439  -0.3321029 ]] 

[[ 9.999976e-01  7.553071e-04 -2.035826e-03 -8.086759e-01]
 [-7.854027e-04  9.998898e-01 -1.482298e-02  3.195559e-01]
 [ 2.024406e-03  1.482454e-02  9.998881e-01 -7.997231e-01]] 



# # pcd to npy
---

In [6]:
# device = o3d.core.Device('cuda:1')
pcd_f = o3d.t.io.read_point_cloud('/data/NIA50/data/2-050_sensor_sample/lidar/1639543825.777566671.pcd')
print(pcd_f)

positions = pcd_f.point.positions.numpy()
intensity = pcd_f.point.intensity.numpy()
pcd = np.concatenate((positions, intensity), axis = 1)

pcd

# np.frombuffer(pcd.tobytes(), dtype='float32').reshape(-1, 4) # bytes에서 numpy로 복원

PointCloud on CPU:0 [276640 points (Float32)].
Attributes: intensity (dtype = Float32, shape = {276640, 1}).


array([[ -1.5803028 ,  -9.255083  ,  -2.3824618 ,   6.        ],
       [ -2.5670419 , -16.682653  ,  -2.200407  ,   1.        ],
       [ -3.6694992 , -28.680662  ,  -1.4689597 ,   0.        ],
       ...,
       [ -0.        ,   0.        ,   0.        ,   2.        ],
       [  0.        ,   0.        ,   0.        ,   9.        ],
       [ -0.7640565 ,  38.062027  ,  -0.88586044,   6.        ]],
      dtype=float32)

In [5]:
import pickle as pkl

custom_infos_train = '/data/NIA50/OpenPCD/data/custom/custom_infos_train.pkl'
with open(custom_infos_train, 'rb') as f:
    train_pkl = pkl.load(f)

train_pkl

[{'point_cloud': {'num_features': 3, 'lidar_idx': '1639543832.078959465'},
  'annos': {'name': array(['Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle',
          'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle',
          'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle',
          'Vehicle', 'Pedestrian', 'Pedestrian', 'Pedestrian', 'Pedestrian',
          'Pedestrian', 'Pedestrian'], dtype='<U10'),
   'gt_boxes_lidar': array([[-3.94200e-01,  6.87450e+00, -1.18780e+00,  3.49770e+00,
            1.99970e+00,  8.40200e-01, -1.62560e+00],
          [-6.85100e-01, -1.67443e+01, -1.32990e+00,  3.47870e+00,
            2.07930e+00,  1.91270e+00, -1.62560e+00],
          [-3.50000e-01,  1.56926e+01, -1.00890e+00,  4.11530e+00,
            1.97400e+00,  9.24300e-01, -1.62560e+00],
          [-2.85680e+00,  2.95184e+01, -1.04610e+00,  4.50350e+00,
            1.73860e+00,  2.09220e+00, -1.62560e+00],
          [-3.27530e+00,  2.44558e+01, -1.34810

# # 특수환경 자율주행 3D 이미지
---

## # make dataframe
---

In [ ]:
# 검색용 코드
df.loc[df['xyzlwh'].apply(lambda x: -31.16 in x[:3])]

### # training
---

In [253]:
train_df = pd.DataFrame(columns=['clip', 'frame', 'xyxy', 'xyzlwh', 'theta', 'eulerangle', 'translation', 'fxfycxcy', 'k1k2p1p2', 'pcd', 'class'])
idx = 0

path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/training/'
clips = sorted([i for i in os.listdir(path) if 'drive_' in i])
# clip = clips[20]

for clip in clips:
    try:
        with open(path+clip+'/calib.txt', 'r') as f:
            calib = [re.sub('\n', '', i) for i in f.readlines()]

        eulerangle = calib[4].split(',')
        translation = calib[6].split(',')
        intrinsic = calib[8].split(',')
        fxfycxcy = intrinsic[:4]
        k1k2p1p2 = intrinsic[4:]

        metas = sorted(os.listdir(path+clip+'/meta/'))

        for meta in metas:
            meta = path+clip+f'/meta/{meta}'
            frame = meta[-11:-5]

            pcd_f = o3d.t.io.read_point_cloud(path+clip+f'/lidar/{clip}_{frame}.pcd')
            positions = pcd_f.point.positions.numpy()
            intensity = pcd_f.point.intensity.numpy()
            pcd = np.concatenate((positions, intensity), axis = 1)

            with open(meta, 'r') as f:
                meta_js = json.load(f)

            objects = meta_js['OBJECT_LIST'][0]['3D_LIST']
            for i in range(len(objects)):
                object = objects[i]
                xyxy = object['BOX']
                xyz = object['LOCATION']
                hwl = object['DIMENSION']
                xyzlwh = xyz + hwl[2:3] + hwl[1:2] + hwl[0:1]
                # xyzlwh = ', '.join(map(str, xyzlwh))
                theta = object['YAW']
                class_ = object['TYPE']

                train_df.loc[idx] = [clip, frame, xyxy, xyzlwh, theta, eulerangle, translation, fxfycxcy, k1k2p1p2, pcd, class_]

                idx+=1
    except:
        pass
    
    train_df.to_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/train_df.csv', index=False)

## # save npy
---

In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/ImageSets'



### # validation
---

In [46]:
val_df = pd.DataFrame(columns=['clip', 'frame', 'xyxy', 'xyzlwh', 'theta', 'eulerangle', 'translation', 'fxfycxcy', 'k1k2p1p2', 'class', 'type'])
idx = 0

path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/'
clips = sorted([i for i in os.listdir(path) if 'drive_' in i])
# clip = clips[20]

for clip in clips:
    try:
        with open(path+clip+'/calib.txt', 'r') as f:
            calib = [re.sub('\n', '', i) for i in f.readlines()]

        eulerangle = list(map(float, calib[4].split(',')))
        translation = list(map(float, calib[6].split(',')))
        intrinsic = list(map(float, calib[8].split(',')))
        fxfycxcy = intrinsic[:4]
        k1k2p1p2 = intrinsic[4:]

        metas = sorted(os.listdir(path+clip+'/lidar/'))

        for meta in metas:
            meta = path+clip+f'/meta/{meta}'
            frame = meta[-11:-5]

            # pcd_f = o3d.t.io.read_point_cloud(path+clip+f'/lidar/{clip}_{frame}.pcd')
            # positions = pcd_f.point.positions.numpy()
            # intensity = pcd_f.point.intensity.numpy()
            # pcd = np.concatenate((positions, intensity), axis = 1).tobytes()

            with open(meta, 'r') as f:
                meta_js = json.load(f)

            objects = meta_js['OBJECT_LIST'][0]['3D_LIST']
            for i in range(len(objects)):
                object = objects[i]
                xyxy = object['BOX']
                xyz = object['LOCATION']
                hwl = object['DIMENSION']
                xyzlwh = xyz + hwl[2:3] + hwl[1:2] + hwl[0:1]
                # xyzlwh = ', '.join(map(str, xyzlwh))
                theta = object['YAW']
                class_ = object['CLASS']
                type_ = object['TYPE']

                val_df.loc[idx] = [clip, frame, xyxy, xyzlwh, theta, eulerangle, translation, fxfycxcy, k1k2p1p2, class_, type_]

                idx+=1
    except:
        pass
    
    val_df.to_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv', index=False)

In [ ]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
val_df.head()

,clip,frame,xyxy,xyzlwh,theta,eulerangle,translation,fxfycxcy,k1k2p1p2,class
0,drive_1151,000000,"[1113, 576, 1126, 611]","[0.0, 0.0, 0.0, 0, 0, 0]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",AUTO_BICYCLE
1,drive_1151,000001,"[1112, 573, 1125, 608]","[0.0, 0.0, 0.0, 0, 0, 0]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",AUTO_BICYCLE
2,drive_1151,000002,"[1112, 573, 1125, 608]","[0.0, 0.0, 0.0, 0, 0, 0]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",AUTO_BICYCLE
3,drive_1151,000003,"[1112, 573, 1125, 608]","[0.0, 0.0, 0.0, 0, 0, 0]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",AUTO_BICYCLE
4,drive_1151,000004,"[1111, 574, 1124, 609]","[0.0, 0.0, 0.0, 0, 0, 0]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",AUTO_BICYCLE


In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/'
save_path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points'

true_data = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
clips = true_data['clip'].unique()

for clip in clips:
    frames = true_data.loc[true_data['clip'] == clip, 'frame'].unique()

    for frame in frames:
        try:
            pcd_name = pcd_name = clip + f'_{frame}.pcd'
            pcd_f = o3d.t.io.read_point_cloud(path+clip+'/lidar/'+pcd_name)

            positions = pcd_f.point.positions.numpy()
            intensity = pcd_f.point.intensity.numpy()
            pcd = np.concatenate((positions, intensity), axis = 1)

            npy = np.save(save_path+f'/{clip}_{frame}.npy', pcd)
        except:
            pass

## # dividing train, val
---

### # imagesets
---

In [4]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
tdf = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
tdf.head()

,clip,frame,xyxy,xyzlwh,theta,eulerangle,translation,fxfycxcy,k1k2p1p2,class,type
19,drive_1151,000019,"[-5, 631, 231, 830]","[-5.18, -6.52, -1.04, 4.44, 1.76, 1.81]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",VEHICLE,SEDAN
20,drive_1151,000020,"[0, 614, 280, 837]","[-5.73, -6.67, -1.03, 4.44, 1.75, 1.8]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",VEHICLE,SEDAN
21,drive_1151,000021,"[-8, 604, 314, 845]","[-6.36, -6.66, -1.02, 4.44, 1.75, 1.79]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",VEHICLE,SEDAN
22,drive_1151,000022,"[-2, 607, 352, 858]","[-6.92, -6.65, -1.01, 4.44, 1.75, 1.78]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",VEHICLE,SEDAN
23,drive_1151,000023,"[-3, 606, 390, 843]","[-7.55, -6.64, -1.0, 4.44, 1.75, 1.77]",0.0,"[-6.308843, -0.01, 0.0]","[0.02, -0.02, -0.03]","[1067.249617, 1070.402406, 967.123811, 600.618...","[-0.156654, 0.10625, 0.0, 0.0]",VEHICLE,SEDAN


In [13]:
images = []

clips = tdf['clip'].unique()
for clip in clips:

    frames = tdf.loc[tdf['clip'] == clip, 'frame'].unique()
    for frame in frames:
        image = clip+f'_{frame}'
        images.append(image)

len(images), len(os.listdir('/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points'))

(20359, 20269)

In [174]:
images = sorted([re.sub('.npy', '', i) for i in os.listdir('/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points')])
len(images)

20269

In [38]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/ImageSets/'

train, val = train_test_split(images, test_size=0.2, random_state=0)

with open(path+'train.txt', 'w') as f:
    f.write('\n'.join(sorted(train)))

with open(path+'val.txt', 'w') as f:
    f.write('\n'.join(sorted(val)))

### # labels
---

In [ ]:
# 차집합

set(labels) - set(images)
set(labels).difference(set(images))

In [54]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/labels/'
labels = [re.sub('.txt', '', i) for i in os.listdir(path)]

for n_pcd in set(labels) - set(images):
    if os.path.exists(path+f'{n_pcd}.txt'):
        os.remove(path+f'{n_pcd}.txt')

### # calibration
---

In [319]:
from scipy.spatial.transform import Rotation as R


path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/calib/'

clips = tdf['clip'].unique()
for clip in clips:

    frames = tdf.loc[tdf['clip'] == clip, 'frame'].unique()
    for frame in frames:
        image = clip+f'_{frame}'

        fxfycxcy = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'fxfycxcy'].values[0])
        eulerangle = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'eulerangle'].values[0])
        translation = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'translation'].values[0])
        intrinsic = np.asarray([[fxfycxcy[0], 0, fxfycxcy[2]], [0, fxfycxcy[1], fxfycxcy[3]], [0, 0, 1]])
        rotation_matrix = R.from_euler('xyz', eulerangle, degrees=True).as_matrix()

        p0 = np.hstack([intrinsic, np.zeros((3, 1))])
        R0_rect = np.eye(3)
        Tr_velo_to_cam = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])
        Tr_imu_to_velo = np.zeros((12))

        # to string
        p0 = ' '.join(map(str, p0.reshape(-1).tolist()))
        p1 = p0
        p2 = p0
        p3 = p0
        R0_rect = ' '.join(map(str, R0_rect.reshape(-1).tolist()))
        Tr_velo_to_cam = ' '.join(map(str, Tr_velo_to_cam.reshape(-1).tolist()))
        Tr_imu_to_velo = ' '.join(map(str, Tr_imu_to_velo.reshape(-1).tolist()))

        label_lines = ['P0: '+p0, 'P1: '+p1, 'P2: '+p2, 'P3: '+p3, 'R0_rect: '+R0_rect, 'Tr_velo_to_cam: '+Tr_velo_to_cam, 'Tr_imu_to_velo: '+Tr_imu_to_velo]
        with open(path + f'{image}.txt', 'w') as f:
            f.write('\n'.join(label_lines))

            # f.write(f'P0: {p0}\n')
            # f.write(f'P1: {p1}\n')
            # f.write(f'P2: {p2}\n')
            # f.write(f'P3: {p3}\n')
            # f.write(f'R0_rect: {R0_rect}\n')
            # f.write(f'Tr_velo_to_cam: {Tr_velo_to_cam}\n')
            # f.write(f'Tr_imu_to_velo: {Tr_imu_to_velo}')

In [320]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/calib/'
labels = [re.sub('.txt', '', i) for i in os.listdir(path)]

for n_pcd in set(labels) - set(images):
    if os.path.exists(path+f'{n_pcd}.txt'):
        os.remove(path+f'{n_pcd}.txt')